In [10]:
import warnings
warnings.filterwarnings(action = "ignore")

import pickle
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

c_F = pickle.load(open("imagedata/Desk/gray_train2_F.pickle", "rb"))
c_T = pickle.load(open("imagedata/Desk/gray_train2_T.pickle", "rb"))

# F = pickle.load(open("imagedata/Tent/gray_train2_tent_F.pickle", "rb"))
# T = pickle.load(open("imagedata/Tent/gray_train2_tent_T.pickle", "rb"))

In [11]:
x = np.concatenate( (c_F, c_T), axis = 0 )


y = [1 for i in range(len(c_F))] + [0 for i in range(len(c_T))] # + [1 for i in range(len(F))] + [0 for i in range(len(T))]

X = []

for rows in x:
    tmp = cv2.resize(rows, (227, 227))
    X.append(tmp)
    
X = np.array(X)
X = X.reshape(-1, 227, 227, 1)

In [12]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.8)
sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))

In [13]:
## Normalization
X = X / 255.0

y = np.array(y)

nums = [i for i in range(8000)]

import random

random.seed(4)
random.shuffle(nums)

X = X[nums]
y = y[nums]

In [14]:
INPUT_SHAPE = (227, 227, 1)

model = Sequential()

# 1st Convolutional Layer
model.add( Conv2D(filters=96, input_shape=(INPUT_SHAPE), kernel_size=(11,11), strides=(4,4), padding='valid') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )

# 2nd Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )

# 3rd Convolutional Layer
model.add( Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# 4th Convolutional Layer
model.add( Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# 5th Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )



########################## Passing it to a Fully Connected layer ##########################
model.add( Flatten() )

K = INPUT_SHAPE[0] * INPUT_SHAPE[1] * INPUT_SHAPE[2]

# 1st Fully Connected Layer
model.add( Dense(4096, input_shape=( K, )) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# 2nd Fully Connected Layer
model.add( Dense(4096) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# 3rd Fully Connected Layer
model.add( Dense(1000) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# Output Layer
model.add( Dense(2) )
model.add( Activation('softmax') )
          
opt = tf.keras.optimizers.Adam(lr = 1e-6)

model.compile(loss = "sparse_categorical_crossentropy", # or  # binary_crossentropy : Dense(1) & Relu
             optimizer = opt,
             metrics = ['accuracy'])

model.summary()

model.fit(X, y, batch_size = 8, epochs = 10, validation_split = 0.1)  # batch_size

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        11712     
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_1 (Activation)    (None, 27, 27, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
____

In [16]:
model.save("tmp8000.model")